In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import math
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
df_train = pd.read_csv('./train_data.csv')
key_train = df_train['SK_ID_CURR']
df_train.describe()
df_train['TARGET'].value_counts()

0    169611
1     14895
Name: TARGET, dtype: int64

In [3]:
df_test = pd.read_csv('./test_data.csv')
key_test = df_test['SK_ID_CURR']
df_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,367294,Cash loans,F,N,Y,0,180000.0,265306.5,25317.0,252000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,2.0,5.0
1,439847,Cash loans,F,N,Y,0,202500.0,346500.0,21069.0,346500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
2,380562,Cash loans,M,Y,N,0,360000.0,545040.0,36553.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,5.0
3,407238,Cash loans,F,N,Y,0,135000.0,307557.0,20682.0,265500.0,...,0,0,0,0,0.0,0.0,1.0,0.0,1.0,2.0
4,239910,Cash loans,F,N,Y,0,157500.0,1056447.0,31018.5,922500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123000,128638,Cash loans,F,Y,Y,1,117000.0,675000.0,32602.5,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
123001,169821,Cash loans,F,N,Y,0,225000.0,1006920.0,42790.5,900000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,0.0
123002,442166,Cash loans,F,N,Y,0,153000.0,625536.0,26851.5,540000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
123003,301605,Cash loans,M,N,Y,0,180000.0,1089000.0,31972.5,1089000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0


### Dropping columns with null values more thant 70%

In [4]:
df_train=df_train.drop(['SK_ID_CURR'],axis=1)
y=df_train['TARGET']
df_train=df_train.drop('TARGET',axis=1)
null_percentage = df_train.isnull().sum()/df_train.shape[0]
col_to_drop = null_percentage[null_percentage>=0.7].keys()
df_train = df_train.drop(col_to_drop, axis=1)
df_train.shape

(184506, 120)

In [5]:
df_test=df_test.drop(['SK_ID_CURR'],axis=1)
df_test = df_test.drop(col_to_drop, axis=1)
df_test.shape

(123005, 120)

In [6]:
df_train.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,184506.000000,1.845060e+05,1.845060e+05,184500.000000,1.843390e+05,184506.000000,184506.000000,184506.000000,184506.00000,184506.000000,...,184506.000000,184506.000000,184506.000000,184506.000000,159508.000000,159508.000000,159508.000000,159508.000000,159508.000000,159508.000000
mean,0.416463,1.689985e+05,5.992854e+05,27114.455805,5.386926e+05,0.020868,-16036.718307,63838.583271,-4984.89506,-2994.113948,...,0.007870,0.000629,0.000553,0.000325,0.006338,0.007335,0.034400,0.267247,0.266770,1.897504
std,0.719614,2.942453e+05,4.021982e+05,14468.652072,3.692263e+05,0.013818,4361.484692,141295.297587,3524.99741,1509.959416,...,0.088362,0.025066,0.023506,0.018030,0.083743,0.114671,0.204187,0.919465,0.894563,1.864439
min,0.000000,2.610000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25201.000000,-17912.000000,-23416.00000,-6383.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.125000e+05,2.700000e+05,16555.500000,2.385000e+05,0.010006,-19677.000000,-2757.000000,-7486.00000,-4299.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.485000e+05,5.155290e+05,24939.000000,4.500000e+05,0.018850,-15747.000000,-1211.000000,-4506.00000,-3256.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12416.000000,-289.000000,-2000.00000,-1721.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,0.00000,0.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,7.000000,24.000000,261.000000,25.000000


### Replacing higher values of 'DAYS_EMPLOYED' with NULL

In [7]:
df_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
df_test['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

### Removing outliers

In [8]:
# OUTLIERS


for col in df_train.columns:
        if ( col!='TARGET' and ((df_train[col].dtype)=='float64') | ((df_train[col].dtype)=='int64')):
            min_th,max_th = df_train[col].quantile([0.1,0.9])
            df_train[col][df_train[col] <= min_th] = min_th
            df_train[col][df_train[col] >= max_th] = max_th
        else:
            df_train[col]=df_train[col]

In [9]:

for col in df_test.columns:
        if (((df_test[col].dtype)=='float64') | ((df_test[col].dtype)=='int64')):
            min_th,max_th = df_test[col].quantile([0.1,0.9])
            df_test[col][df_test[col] <= min_th] = min_th
            df_test[col][df_test[col] >= max_th] = max_th
        else:
            df_test[col]=df_test[col]

In [10]:
df_train.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,184506.000000,184506.000000,1.845060e+05,184500.000000,1.843390e+05,184506.000000,184506.000000,151270.000000,184506.000000,184506.000000,...,184506.0,184506.0,184506.0,184506.0,159508.0,159508.0,159508.0,159508.000000,159508.000000,159508.000000
mean,0.300391,159463.528725,5.708827e+05,26244.187122,5.190314e+05,0.019614,-16023.824770,-2138.017915,-4835.156223,-2999.894377,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163685,0.191257,1.700648
std,0.458429,61194.013563,3.196381e+05,11211.929904,2.979188e+05,0.009915,4023.667665,1631.695380,3108.419602,1405.197701,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.369990,0.393292,1.436219
min,0.000000,81000.000000,1.800000e+05,11078.550000,1.800000e+05,0.006629,-22176.000000,-5334.000000,-9930.000000,-4723.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,0.000000,112500.000000,2.700000e+05,16555.500000,2.385000e+05,0.010006,-19677.000000,-3176.000000,-7486.000000,-4299.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
50%,0.000000,148500.000000,5.155290e+05,24939.000000,4.500000e+05,0.018850,-15747.000000,-1646.000000,-4506.000000,-3256.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000
75%,1.000000,202500.000000,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12416.000000,-764.000000,-2000.000000,-1721.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.000000
max,1.000000,270000.000000,1.133748e+06,45954.000000,1.093500e+06,0.035792,-10296.000000,-333.000000,-687.000000,-726.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,4.000000


In [11]:
# SEPARATING NUMERICAL AND CATEGORICAL COLUMNS
categorical = df_train.select_dtypes(include=['object']).columns.tolist()
numerical = df_train.select_dtypes(exclude=['object']).columns.tolist()

df_train.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,184506.000000,184506.000000,1.845060e+05,184500.000000,1.843390e+05,184506.000000,184506.000000,151270.000000,184506.000000,184506.000000,...,184506.0,184506.0,184506.0,184506.0,159508.0,159508.0,159508.0,159508.000000,159508.000000,159508.000000
mean,0.300391,159463.528725,5.708827e+05,26244.187122,5.190314e+05,0.019614,-16023.824770,-2138.017915,-4835.156223,-2999.894377,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163685,0.191257,1.700648
std,0.458429,61194.013563,3.196381e+05,11211.929904,2.979188e+05,0.009915,4023.667665,1631.695380,3108.419602,1405.197701,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.369990,0.393292,1.436219
min,0.000000,81000.000000,1.800000e+05,11078.550000,1.800000e+05,0.006629,-22176.000000,-5334.000000,-9930.000000,-4723.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,0.000000,112500.000000,2.700000e+05,16555.500000,2.385000e+05,0.010006,-19677.000000,-3176.000000,-7486.000000,-4299.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
50%,0.000000,148500.000000,5.155290e+05,24939.000000,4.500000e+05,0.018850,-15747.000000,-1646.000000,-4506.000000,-3256.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000
75%,1.000000,202500.000000,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12416.000000,-764.000000,-2000.000000,-1721.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.000000
max,1.000000,270000.000000,1.133748e+06,45954.000000,1.093500e+06,0.035792,-10296.000000,-333.000000,-687.000000,-726.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,4.000000


In [12]:
for cols in categorical:
    df_train[cols] = df_train[cols].fillna(df_train[cols].mode()[0])

In [13]:
for cols in categorical:
    df_test[cols] = df_test[cols].fillna(df_test[cols].mode()[0])

### Replacing null values in numerical columns with median

In [14]:
for cols in numerical:
    df_train[cols] = df_train[cols].fillna(df_train[cols].median())
# df_train.describe()`

In [15]:
for cols in numerical:
    df_test[cols] = df_test[cols].fillna(df_test[cols].median())

### Standardization

In [16]:
for cols in numerical:
    if(df_train[cols].max()>1):
        df_train[cols] = (df_train[cols] - df_train[cols].min()) / (df_train[cols].max() - df_train[cols].min())
        
df_train.describe()


,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,...,184506.0,184506.0,184506.0,184506.0,184506.0,184506.0,184506.0,184506.000000,184506.000000,184506.000000
mean,0.300391,0.415151,0.409839,0.434850,0.371066,0.019614,-16023.824770,-2049.388237,-4835.156223,-2999.894377,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.141508,0.165344,0.401430
std,0.458429,0.323778,0.335139,0.321480,0.325989,0.009915,4023.667665,1489.490723,3108.419602,1405.197701,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.348545,0.371492,0.339186
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.006629,-22176.000000,-5334.000000,-9930.000000,-4723.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,0.000000,0.166667,0.094365,0.157043,0.064039,0.010006,-19677.000000,-2757.000000,-7486.000000,-4299.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.250000
50%,0.000000,0.357143,0.351800,0.397427,0.295567,0.018850,-15747.000000,-1646.000000,-4506.000000,-3256.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.250000
75%,1.000000,0.642857,0.659136,0.674327,0.546798,0.028663,-12416.000000,-930.000000,-2000.000000,-1721.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.035792,-10296.000000,-333.000000,-687.000000,-726.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000


In [17]:
for cols in numerical:
    if(df_test[cols].max()>1):
        df_test[cols] = (df_test[cols] - df_test[cols].min()) / (df_test[cols].max() - df_test[cols].min())

df_test.describe()


,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,123005.000000,123005.000000,123005.000000,123005.000000,123005.000000,123005.000000,123005.000000,123005.000000,123005.000000,123005.000000,...,123005.0,123005.0,123005.0,123005.0,123005.0,123005.0,123005.0,123005.000000,123005.000000,123005.000000
mean,0.200386,0.415034,0.410318,0.433336,0.370080,0.019597,-16023.704402,-2053.265664,-4839.398561,-3000.102378,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.143490,0.163148,0.401998
std,0.333436,0.324479,0.335871,0.321260,0.325927,0.009900,4032.337109,1488.005515,3103.207662,1402.634389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.350573,0.369502,0.339176
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.006629,-22187.000000,-5332.000000,-9935.000000,-4724.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,0.000000,0.166667,0.094787,0.156101,0.064039,0.010006,-19689.000000,-2764.000000,-7471.000000,-4298.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.250000
50%,0.000000,0.333333,0.350014,0.394558,0.295567,0.018850,-15753.000000,-1650.000000,-4501.000000,-3253.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.250000
75%,0.500000,0.642857,0.662085,0.672714,0.546798,0.028663,-12409.000000,-936.000000,-2028.000000,-1718.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.035792,-10265.000000,-334.000000,-695.000000,-736.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000


### Dropping columns which have max=0 after scaling

In [18]:
drop_cols=[]

for col in df_train.columns:
        if (((df_train[col].dtype)=='float64') | ((df_train[col].dtype)=='int64')):
            if(df_train[col].max()==0):
                drop_cols.append(col)

drop_cols
df_train=df_train.drop(drop_cols,axis=1)
df_test=df_test.drop(drop_cols,axis=1)

### Label encoding

In [19]:
# LABEL ENCODING
categorical = df_train.select_dtypes(include=['object']).columns.tolist()
df_train[categorical] = df_train[categorical].apply(LabelEncoder().fit_transform)
df_train.describe()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,FLAG_DOCUMENT_3,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,...,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000
mean,0.094810,0.342271,0.340780,0.694064,0.300391,0.415151,0.409839,0.434850,0.371066,5.222687,...,0.079865,4.084252,0.007821,0.280116,0.114760,0.277584,0.708389,0.141508,0.165344,0.401430
std,0.292953,0.474494,0.473973,0.460804,0.458429,0.323778,0.335139,0.321480,0.325989,1.772521,...,0.050096,0.925124,0.088089,0.367916,0.318734,0.366447,0.454505,0.348545,0.371492,0.339186
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.094365,0.157043,0.064039,6.000000,...,0.066800,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,0.357143,0.351800,0.397427,0.295567,6.000000,...,0.068700,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.250000
75%,0.000000,1.000000,1.000000,1.000000,1.000000,0.642857,0.659136,0.674327,0.546798,6.000000,...,0.070300,4.000000,0.000000,0.500000,0.000000,0.500000,1.000000,0.000000,0.000000,0.750000
max,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,...,0.226700,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
# LABEL ENCODING
categorical = df_test.select_dtypes(include=['object']).columns.tolist()
df_test[categorical] = df_test[categorical].apply(LabelEncoder().fit_transform)

### LGBM

In [21]:
# lgbm = lgb.LGBMClassifier(random_state = 50, n_jobs = -1, class_weight = 'balanced')
lgbm = lgb.LGBMClassifier(**{'reg_lambda': 0.1, 
                        'reg_alpha': 0.2, 
                        'num_leaves': 100, 
                        'n_estimators': 300,
                        'min_child_samples': 800, 
                        'learning_rate': 0.0701,
                        'max_bin': 500,
                        'objective': 'binary',
                        'n_jobs': -1,
                        'class_weight':'balanced',
                        'n_estimators':1000,
                        'random_state':100})
# lgbm = lgb.LGBMClassifier(learning_rate=0.05, boosting_type='gbdt',objective='binary',num_leaves=100,max_depth=10)

# logisticRegr.get_params()

In [22]:
X_test = df_test
df_train.shape

(184506, 91)

In [23]:
lgbm.fit(df_train, y, eval_metric = 'auc')
ytrain_pred = lgbm.predict(df_train)
lgbm.fit(df_train,ytrain_pred,eval_metric = 'auc')
y_pred = lgbm.predict(X_test)

In [24]:
df_pred = pd.DataFrame(y_pred, columns = ['TARGET'])
df_pred.insert(0, "SK_ID_CURR", key_test)
df_pred.to_csv('LgbPredictions.csv')
zero=0
one=0
for i in y_pred:
    if i==0:
        zero=zero+1
    else:
        one=one+1
print(zero,one)

105042 17963
